In [25]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels

## Load DATA


### Observations


In [26]:
papa_obs = xr.open_dataset("./products/Papa_obs.zarr", engine="zarr")
papa_obs

<xarray.Dataset> Size: 4MB
Dimensions:    (time: 623, latitude: 12, longitude: 31)
Coordinates:
  * latitude   (latitude) float64 96B 46.5 47.5 48.5 49.5 ... 55.5 56.5 57.5
  * longitude  (longitude) float64 248B -157.5 -156.5 -155.5 ... -128.5 -127.5
  * time       (time) datetime64[ns] 5kB 1995-09-24 1995-09-25 ... 2020-08-31
Data variables:
    day        (time, latitude, longitude) float64 2MB ...
    night      (time, latitude, longitude) float64 2MB ...

In [28]:
papa_forcing = xr.open_dataset(
    "/Users/adm-lehodey/Documents/Workspace/Data/phd/FORCING/Calcofi_forcing.zarr", engine="zarr"
)
papa_forcing

<xarray.Dataset> Size: 3GB
Dimensions:              (time: 9436, depth: 3, latitude: 55, longitude: 103)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 440B 0.5 1.5 2.5 ... 52.5 53.5 54.5
  * longitude            (longitude) float64 824B -179.5 -178.5 ... -78.5 -77.5
  * time                 (time) datetime64[ns] 75kB 1998-01-01 ... 2023-11-01
Data variables:
    T                    (time, depth, latitude, longitude) float64 1GB ...
    npp                  (time, latitude, longitude) float64 428MB ...
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 1GB ...
    zooc                 (time, latitude, longitude) float64 428MB ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

In [37]:
common_latitudes = sorted(set(papa_obs.latitude.data).intersection(set(papa_forcing.latitude.data)))
common_longitudes = sorted(set(papa_obs.longitude.data).intersection(set(papa_forcing.longitude.data)))
common_times = sorted(set(papa_obs.time.data).intersection(set(papa_forcing.time.data)))

print(f"Latitudes communes: {common_latitudes}")
print(f"Temps communs: {len(common_times)} dates, de {common_times[0]} à {common_times[-1]}")
print(f"Longitudes communes: {common_longitudes}")

# Sélection des données de papa_forcing avec les coordonnées communes
papa_forcing_common = papa_forcing.sel(latitude=common_latitudes, longitude=common_longitudes, time=common_times)

# Sélection des données de papa_obs avec les coordonnées communes
papa_obs_common = papa_obs.sel(latitude=common_latitudes, longitude=common_longitudes, time=common_times)

Latitudes communes: [46.5, 47.5, 48.5, 49.5, 50.5, 51.5, 52.5, 53.5, 54.5]
Temps communs: 579 dates, de 1998-01-19T00:00:00.000000000 à 2020-08-31T00:00:00.000000000
Longitudes communes: [-157.5, -156.5, -155.5, -154.5, -153.5, -152.5, -151.5, -150.5, -149.5, -148.5, -147.5, -146.5, -145.5, -144.5, -143.5, -142.5, -141.5, -140.5, -139.5, -138.5, -137.5, -136.5, -135.5, -134.5, -133.5, -132.5, -131.5, -130.5, -129.5, -128.5, -127.5]


# TODO : Erreur, il manque des coordonnées dans les forçages extraits avec COPERNICUS.


In [16]:
papa_forcing.interp_like(papa_obs).dropna(dim="time", how="all").dropna(dim="depth", how="all").dropna(
    dim="latitude", how="all"
).dropna(dim="longitude", how="all")

<xarray.Dataset> Size: 10MB
Dimensions:              (time: 579, depth: 3, latitude: 9, longitude: 31)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * time                 (time) datetime64[ns] 5kB 1998-01-19 ... 2020-08-31
  * latitude             (latitude) float64 72B 46.5 47.5 48.5 ... 53.5 54.5
  * longitude            (longitude) float64 248B -157.5 -156.5 ... -127.5
Data variables:
    T                    (time, depth, latitude, longitude) float64 4MB 7.034...
    npp                  (time, latitude, longitude) float64 1MB 172.4 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 4MB 87.19...
    zooc                 (time, latitude, longitude) float64 1MB 1.596 ... nan
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

In [20]:
papa_obs.dropna(dim="time", how="all").dropna(dim="latitude", how="all").dropna(dim="longitude", how="all")

<xarray.Dataset> Size: 3MB
Dimensions:    (time: 623, latitude: 12, longitude: 26)
Coordinates:
  * latitude   (latitude) float64 96B 46.5 47.5 48.5 49.5 ... 55.5 56.5 57.5
  * longitude  (longitude) float64 208B -156.5 -153.5 -152.5 ... -129.5 -128.5
  * time       (time) datetime64[ns] 5kB 1995-09-24 1995-09-25 ... 2020-08-31
Data variables:
    day        (time, latitude, longitude) float64 2MB nan nan nan ... nan nan
    night      (time, latitude, longitude) float64 2MB nan nan nan ... nan nan

In [ ]:
LATITUDE = papa_obs.latitude.data[0]
LONGITUDE = papa_obs.longitude.data[0]
# The observation data time range is starting too early. We will use the time range of the model data.
# START_TIME = pd.Timestamp(papa_obs.time.data[0]).strftime("%Y-%m-%d")
# END_TIME = pd.Timestamp(papa_obs.time.data[-1]).strftime("%Y-%m-%d")
START_TIME = "1998-01-01"
END_TIME = "2022-12-31"
DELTA_IMPORT = 0.5
DELTA_COMPUTE = 0.1

print(
    f"Latitude: {LATITUDE}\nLongitude: {LONGITUDE}\nStart Time: {START_TIME}\nEnd Time: {END_TIME}\nDelta import: {DELTA_IMPORT}"
)

### CMEMS


In [ ]:
FROM_API = True
path_cmems_product_bio = "..."
path_cmems_product_physic = "..."

In [ ]:
# copernicusmarine.login()

coordinates = {
    "minimum_latitude": LATITUDE - DELTA_IMPORT,
    "maximum_latitude": LATITUDE + DELTA_IMPORT,
    "minimum_longitude": LONGITUDE - DELTA_IMPORT,
    "maximum_longitude": LONGITUDE + DELTA_IMPORT,
    "start_datetime": START_TIME,
    "end_datetime": END_TIME,
}


if FROM_API:
    bio = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i", **coordinates)
    bio.load()
    physic = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i", **coordinates)
    physic.load()
else:
    bio = xr.load_dataset(path_cmems_product_bio)
    physic = xr.load_dataset(path_cmems_product_physic)

In [ ]:
with xr.set_options(keep_attrs=True):
    data = xr.merge([bio, physic])
    lat_attrs, lon_attrs = data.latitude.attrs, data.longitude.attrs
    data = data.sel(
        time=slice(START_TIME, END_TIME),
    ).sel(
        latitude=slice(LATITUDE - DELTA_COMPUTE, LATITUDE + DELTA_COMPUTE),
        longitude=slice(LONGITUDE - DELTA_COMPUTE, LONGITUDE + DELTA_COMPUTE),
    )
    data = data.mean(dim=["latitude", "longitude"], skipna=True)
    data = data.expand_dims({"latitude": [LATITUDE], "longitude": [LONGITUDE]})
    data.latitude.attrs, data.longitude.attrs = lat_attrs, lon_attrs
    data["T"].attrs["units"] = str(StandardUnitsLabels.temperature.units)
data

## Change the depth to be categorical AND starting with 0 value rather than 1


In [ ]:
depth_attrs = {
    "flag_values": "[0 1 2]",
    "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
    "desc": "The layer depth according to Seapodym definition",
    "standard_name": "depth",
    "long_name": "Layer depth",
    "axis": "Z",
}
data = data.assign_coords(depth=[0, 1, 2])
data.depth.attrs = depth_attrs
data

## Gathering Data & Save


In [ ]:
data.to_zarr("./products/Papa_cmems.zarr", mode="w")